In [4]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Authenticate and create pipeline
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
pipe = pipeline("text-generation", model=model_name, use_auth_token=True)

messages = [
    {"role": "user", "content": "Who are you?"},
]
response = pipe(messages)
print(response)

def generate_search_phrases(topic, num_phrases=5, max_length=30):
    """
    Generate search phrases based on the given topic.
    
    Args:
    topic (str): The input topic.
    num_phrases (int): Number of phrases to generate.
    max_length (int): Maximum length of each generated phrase.
    
    Returns:
    list: A list of generated search phrases.
    """
    # Load model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    
    # Step 2: Define input text
    input_text = f"Search phrases about {topic}:"
    
    # Step 3: Tokenize the input
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    
    # Generate text
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_phrases,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )
    
    # Step 4: Decode the generated tokens to form search phrases
    search_phrases = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output]
    
    # Remove the input text from the generated phrases
    search_phrases = [phrase.replace(input_text, "").strip() for phrase in search_phrases]
    
    return search_phrases

# Example usage
topic = "artificial intelligence"
generated_phrases = generate_search_phrases(topic)

print(f"Generated search phrases about {topic}:")
for i, phrase in enumerate(generated_phrases, 1):
    print(f"{i}. {phrase}")


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

C:\Users\aadhi\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aadhi\.cache\huggingface\hub\models--mistralai--Mistral-Nemo-Instruct-2407. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: Failed to import transformers.models.mistral.modeling_tf_mistral because of the following error (look up to see its traceback):
numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

Video file C:\dmvr\examples\train\train_splits\dia125_utt3.mp4 does not exist.


In [12]:
from huggingface_hub import snapshot_download
from pathlib import Path
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

# Setup paths and download model
mistral_models_path = Path.home().joinpath('mistral_models', 'Nemo-Instruct')
mistral_models_path.mkdir(parents=True, exist_ok=True)



snapshot_download(repo_id="mistralai/Mistral-Nemo-Instruct-2407", 
                  allow_patterns=["params.json", "consolidated.safetensors", "tekken.json"], 
                  local_dir=mistral_models_path)

# Load model and tokenizer
tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tekken.json")
model = Transformer.from_folder(mistral_models_path)

def generate_search_phrases(topic, num_phrases=5, max_length=30):
    """
    Generate search phrases based on the given topic.
    
    Args:
    topic (str): The input topic.
    num_phrases (int): Number of phrases to generate.
    max_length (int): Maximum length of each generated phrase.
    
    Returns:
    list: A list of generated search phrases.
    """
    # Define input text
    input_text = f"Search phrases about {topic}:"
    
    # Create ChatCompletionRequest
    completion_request = ChatCompletionRequest(messages=[UserMessage(content=input_text)])
    
    # Tokenize the input
    tokens = tokenizer.encode_chat_completion(completion_request).tokens
    
    # Generate text
    out_tokens, _ = generate([tokens], model, max_tokens=max_length, temperature=0.7, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
    
    # Decode the generated tokens to form search phrases
    search_phrases = tokenizer.decode(out_tokens[0])
    
    # Remove the input text from the generated phrases
    search_phrases = search_phrases.replace(input_text, "").strip().split("\n")[:num_phrases]
    
    return search_phrases

# Example usage
topic = "artificial intelligence"
generated_phrases = generate_search_phrases(topic)

print(f"Generated search phrases about {topic}:")
for i, phrase in enumerate(generated_phrases, 1):
    print(f"{i}. {phrase}")


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 32549: character maps to <undefined>

In [3]:
import json
import chardet
from huggingface_hub import snapshot_download
from pathlib import Path
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

# Setup paths and download model
mistral_models_path = Path.home().joinpath('mistral_models', 'Nemo-Instruct')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-Nemo-Instruct-2407", 
                  allow_patterns=["params.json", "consolidated.safetensors", "tekken.json"], 
                  local_dir=mistral_models_path)

def load_tokenizer(file_path):
    # Read the file in binary mode
    with open(file_path, 'rb') as file:
        raw_data = file.read()

    # Detect the file encoding
    detected = chardet.detect(raw_data)
    print(f"Detected encoding: {detected['encoding']} (confidence: {detected['confidence']})")

    # Try decoding with the detected encoding, falling back to 'utf-8' if it fails
    try:
        file_content = raw_data.decode(detected['encoding'])
    except (UnicodeDecodeError, TypeError):
        print(f"Failed to decode with {detected['encoding']}, falling back to utf-8")
        file_content = raw_data.decode('utf-8', errors='ignore')

    # Create a temporary file with the decoded content
    temp_file = Path(file_path).with_suffix('.temp.json')
    with open(temp_file, 'w', encoding='utf-8') as file:
        file.write(file_content)

    try:
        # Load the tokenizer from the temporary file
        tokenizer = MistralTokenizer.from_file(str(temp_file))
        return tokenizer
    except Exception as e:
        print(f"Error loading tokenizer: {str(e)}")
        raise
    finally:
        # Remove the temporary file
        temp_file.unlink()

tokenizer = load_tokenizer(f"{mistral_models_path}/tekken.json")
model = Transformer.from_folder(mistral_models_path)

def generate_search_phrases(topic, num_phrases=5, max_length=30):
    """
    Generate search phrases based on the given topic.
    
    Args:
    topic (str): The input topic.
    num_phrases (int): Number of phrases to generate.
    max_length (int): Maximum length of each generated phrase.
    
    Returns:
    list: A list of generated search phrases.
    """
    # Define input text
    input_text = f"Search phrases about {topic}:"
    
    # Create ChatCompletionRequest
    completion_request = ChatCompletionRequest(messages=[UserMessage(content=input_text)])
    
    # Tokenize the input
    tokens = tokenizer.encode_chat_completion(completion_request).tokens
    
    # Generate text
    out_tokens, _ = generate([tokens], model, max_tokens=max_length, temperature=0.7, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
    
    # Decode the generated tokens to form search phrases
    search_phrases = tokenizer.decode(out_tokens[0])
    
    # Remove the input text from the generated phrases
    search_phrases = search_phrases.replace(input_text, "").strip().split("\n")[:num_phrases]
    
    return search_phrases

# Example usage
topic = "artificial intelligence"
generated_phrases = generate_search_phrases(topic)
print(f"Generated search phrases about {topic}:")
for i, phrase in enumerate(generated_phrases, 1):
    print(f"{i}. {phrase}")


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Detected encoding: utf-8 (confidence: 0.99)
Error loading tokenizer: 'charmap' codec can't decode byte 0x81 in position 34446: character maps to <undefined>


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 34446: character maps to <undefined>

In [16]:
%pip install chardet

  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
%pip install mistral_inference


  Using cached fire-0.6.0.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/83.5 kB ? eta -:--:--
     -------------- ------------------------- 30.7/83.5 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 83.5/83.5 kB 1.6 MB/s eta 0:00:00
  Using cached sentencepiece-0.2.0-cp39-cp39-win_amd64.whl.metadata (8.3 kB)
  Using cached tiktoken-0.7.0-cp39-cp39-win_amd64.whl.metadata (6.8 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached torch-2.3.1-cp39-cp39-win_amd64.whl.metadata (26 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from huggingface_hub import login

# This will prompt you to enter your token
login()


In [5]:
%pip install transformers


  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.5.15-cp39-cp39-win_amd64.whl.metadata (41 kB)
  Using cached safetensors-0.4.3-cp39-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl.metadata (6.9 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
   ---------------------------------------- 0.0/419.0 kB ? eta -:--:--
   --- ----------------------------------- 41.0/419.0 kB 991.0 kB/s eta 0:00:01
   -------- ------------------------------- 92.2/419.0 kB 1.7 MB/s eta 0:00:01
   --------------------- ------------------ 225.3/419.0 kB 2.3 MB/s eta 0:00:01
   ------------------------ --------------- 256.0/419.0 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------  409.6/419.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 419.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
!pip show transformers

Name: transformers
Version: 4.42.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\aadhi\anaconda3\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: auto_gptq, peft, sentence-transformers
